In [1]:
import tensorflow as tf
import time
import multiprocessing
import numpy as np

In [2]:
class MyTest():
    def __init__(self, thread_pool_size=4):
        self.a=[np.ones((2,2), dtype=np.float32) * i for i in range(1000)]
        self.pool = multiprocessing.pool.ThreadPool(thread_pool_size)
    def __len__(self):
        return len(self.a)
    def __getitem__(self, i):
        #print("loading ", i)
        time.sleep(0.02)
        return self.a[i]
    def getitems(self, indices):
        return np.array(self.pool.map(self.__getitem__ ,indices))


In [3]:
%%timeit
t=MyTest(10)
a=t.getitems(list(range(100)))

267 ms ± 2.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
t=MyTest(10)
tf_fun=lambda i: tf.numpy_function(t.__getitem__, [i] , tf.float32 )

In [ ]:
%%timeit
ds=tf.data.Dataset.range(100).map(tf_fun)
for x in ds:
    pass

2022-08-11 09:20:48.680492: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-11 09:20:48.680602: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jwc02n186.juwels): /proc/driver/nvidia/version does not exist
2022-08-11 09:20:48.682073: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-11 09:20:49.177910: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [8]:
tf_fun2=lambda i: tf.numpy_function(t.getitems, [i] , tf.float32 )

In [9]:
%%timeit
ds=tf.data.Dataset.range(100).batch(10).map(tf_fun2).unbatch()
for x in ds:
    pass

262 ms ± 3.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
